<a href="https://colab.research.google.com/github/etalab-ia/ami-ia/blob/master/notebooks/intro-ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction à l'apprentissage supervisé

Une introduction au *machine learning* avec l'examples des données de contrôles sanitaires de la direction générale de l'alimentation : https://www.data.gouv.fr/fr/datasets/resultats-des-controles-officiels-sanitaires-dispositif-dinformation-alimconfiance/.

### Contexte

La publication des résultats des contrôles sanitaires dans le secteur alimentaire (restaurants, cantines, abattoirs, etc.) est une attente légitime des citoyens qui participe à l’amélioration de la confiance du consommateur. Prévue par la loi d’Avenir pour l’agriculture, l’alimentation et la forêt, du 13 octobre 2014, cette mesure s’inscrit dans une évolution vers une plus grande transparence de l’action de l’État.

Il s’agit de rendre public le résultat des contrôles officiels en sécurité sanitaire des aliments réalisés dans tous les établissements de la chaîne alimentaire : abattoirs, commerces de détail (métiers de bouche, restaurants, supermarchés, marchés, vente à la ferme, etc.), restaurants collectifs et établissements agroalimentaires, depuis le 1er mars 2017.

### Notre objectif

Peut-on anticiper quel sera le résultat d'un nouveau contrôle (satisfaisant ou non) à partir des caractéristiques de l'établissement ?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(123)

%matplotlib inline

## 1. Importation et exploration des données

### Importer un jeu de données

Grâce à la librairie pandas. Par exemple, depuis internet.


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/etalab-ia/ami-ia/master/session2/data/export_alimconfiance.csv',
                 sep=';')

Quelle est la taille de mes données ?

In [ ]:
df.shape

### A quoi ressemble mon jeu de données ?


In [ ]:
df.sample(5)

Quels types de données ?


In [ ]:
df.dtypes

### Explorer le contenu de chaque colonne

Quels résultats pour les contrôles ?

In [ ]:
df['Synthese_eval_sanit'].value_counts()

Quels types d'établissements ?

In [ ]:
df['filtre'].value_counts()

Certaines établissements appartiennent à plusieurs types, mais quelle proportion ?

In [ ]:
(df['filtre'].str.contains('\|')).sum() / len(df)

On fait le choix (arbitraire) de ne garder que le premier type renseigné

In [ ]:
df['filtre'] = df['filtre'].str.split('\|').str[0]

Et à quoi correspond la colonne `ods_type_activite` ?


In [ ]:
df['ods_type_activite'].value_counts()

Où se situent géographiquement les contrôles ?

In [ ]:
df['latitude'] = df['geores'].str.split(',').str[0].astype(float)
df['longitude'] = df['geores'].str.split(',').str[1].astype(float)
df.sample(1)

In [ ]:
!pip install geopandas
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black')
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df['longitude'], df['latitude']))
gdf.plot(ax=base)

Exploration de l'étrange colonne `Agrement`

In [ ]:
df['Agrement'].count() / len(df['Agrement'])

## 2. Feature engineering

Le feature engineerng consiste choisir quelles features conserver, en ajouter de nouvelles, et les transformer sous un format adapté pour l'entraînement des modèles. Contrairement à ce qu'on peut penser, **c'est la phase la plus complexe et la plus longue.** Elle requiert une connaissance métier, une expérience dans la création de ces features, et beaucoup d'imagination !

A garder en tête : les algorithmes d'apprentissage, supervisés ou non, **ne savent travailler qu'avec des données numériques.**

#### Simplifions notre valeur cible sous un format numérique

In [ ]:
simplification = {'Très satisfaisant': 1,
                  'Satisfaisant': 1,
                  'A améliorer': 0,
                  'A corriger de manière urgente': 0}
df['Synthese_eval_sanit'] = df['Synthese_eval_sanit'].map(simplification)
df['Synthese_eval_sanit'].value_counts()

### Créer de nouvelles features

Utilisons l'agrément comme un marqueur de qualité

In [ ]:
df['has_agrement'] = pd.notnull(df['Agrement']).astype(int)
df['has_agrement'].sample(1)

Ne serait-il pas possible de prendre en compte les contrôles passés d'un restaurant ?

In [ ]:
df['SIRET'].value_counts().hist(bins=100)

In [ ]:
df['count_controls_siret'] = df.groupby('SIRET')['Synthese_eval_sanit'].transform(lambda x: x.count())

On peut aussi utiliser de l'information géographique, mais le niveau du code postal est trop précis.
Prenons plutôt les codes des départements.

In [ ]:
df['Code_postal'].nunique()

In [ ]:
sum(df['Code_postal'].isnull())

In [ ]:
df = df[df['Code_postal'].notnull()]
df['dept'] = np.floor(df['Code_postal'] / 1000)
df['dept'].sample(1)

On dispose de la date d'inspection : on peut donc en tirer de nouvelles features temporelles.

In [ ]:
df['Date_inspection'].sample(1)

In [ ]:
df['Date_inspection'] = pd.to_datetime(df['Date_inspection'],
                                       format='%Y-%m-%dT%H:%M:%S', utc=True)
df['Date_inspection'].sample(1)

In [ ]:
df['year'] = df['Date_inspection'].dt.year
df['month'] = df['Date_inspection'].dt.month
df['weekday'] = df['Date_inspection'].dt.weekday

Peut-on utiliser des informations transverses à plusieurs zones ou groupes ?

- Par département

In [ ]:
df['count_controls_dept'] = df.groupby('dept')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_dept'] = df.groupby('dept')['Synthese_eval_sanit'].transform(lambda x: x.mean())

- Par secteur d'activité (Restaurant, Boucherie-Charcuterie, Boulangerie-Pâtisserie, etc.)

In [ ]:
df['filtre'] = df['filtre'].fillna('NA')
df['count_controls_filtre'] = df.groupby('filtre')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_filtre'] = df.groupby('filtre')['Synthese_eval_sanit'].transform(lambda x: x.mean())

- Par type de produits vendus (Viandes et produits carnés, 
Produits de la mer et d'eau douce, Lait et produits laitiers, etc.)

In [ ]:
df['ods_type_activite'] = df['ods_type_activite'].fillna('NA')
df['count_controls_activite'] = df.groupby('ods_type_activite')['Synthese_eval_sanit'].transform(
                                                                                   lambda x: x.count())
df['score_controls_activite'] = df.groupby('ods_type_activite')['Synthese_eval_sanit'].transform(
                                                                                   lambda x: x.mean())

- Par jour de la semaine !

In [ ]:
df['count_controls_wday'] = df.groupby('weekday')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_wday'] = df.groupby('weekday')['Synthese_eval_sanit'].transform(lambda x: x.mean())

A quoi ressemblent nos ajouts ?

In [ ]:
df.sample(1)

### Supprimer des features inutiles ou inutilisables

In [ ]:
drop_cols = ['APP_Libelle_etablissement', 'Code_postal', 'SIRET', 'Libelle_commune',
             'APP_Libelle_activite_etablissement', 'Numero_inspection', 'Date_inspection',
             'Agrement', 'geores', 'ods_adresse', 'geometry']
df = df.drop(drop_cols, axis=1)

### Encoder les colonnes catégorielles

Il est nécessaire de transformer les catégories en nombres. Mais comment faire ?

| Etablissement | Activité           |
| :------ | :-------------     | 
| A       | Restaurant         |
| B       | Producteur Fermier |
| C       | Boucherie          |
| D       | Restaurant         |

devient :

| Etablissement | Activité_Restaurant | Activité_ProducteurFermier | Activité_Boucherie |
| :------ | :-: | :-: | :-: | 
| A       | 1 | 0 | 0 |
| B       | 0 | 1 | 0 |
| C       | 0 | 0 | 1 |
| D       | 1 | 0 | 0 |


- **Etape 1 :** on créer un encodeur, qui va directement transformer toutes les colonnes catégorielles

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_features = ['dept', 'filtre', 'ods_type_activite', 'month', 'weekday']
encoder = OneHotEncoder(drop='first', sparse=False).fit(df[categorical_features])

- **Etape 3 :** on fusionne les encodages avec le dataframe originel

In [ ]:
df = pd.concat([df,
                pd.DataFrame(data=encoder.transform(df[categorical_features]), 
                             columns=encoder.get_feature_names(categorical_features), 
                             index=df.index)], axis=1)
df.shape

In [ ]:
df.sample(1)

## 3. Entraîner et comparer des modèles

Une fois que nos données sont sélectionnées et mises en forme, la phase d'apprentissage consiste à choisir le modèle et à optimiser ses paramètres.

### Un dernier nettoyage du jeu de données

In [ ]:
df.shape

Suppression des lignes contenant des NaN

In [ ]:
df = df.dropna()
df.shape

Suppression des colonnes avec une seule valeur différente

In [ ]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col, inplace=True, axis=1)
df.shape

### Séparation en jeux de données "train" et "test"

In [ ]:
from sklearn.model_selection import train_test_split
features = [col for col in df.columns if col not in categorical_features
            and col != 'Synthese_eval_sanit']

df_train, df_test = train_test_split(df, test_size=0.2)

X_train = df_train[features]
y_train = df_train['Synthese_eval_sanit']

X_test = df_test[features]
y_test = df_test['Synthese_eval_sanit']

### Echantillonner pour équilibrer les classes


In [ ]:
y_train.mean(), y_test.mean()

Les deux catégories que nous cherchons à prédire ne sont pas équilibrées ! L'algorithme pourrait utiliser cette faille pour facilement obtenir de bons résultats.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler()
X_train, y_train = sampler.fit_resample(X_train, y_train)
X_test, y_test = sampler.fit_resample(X_test, y_test)

y_train.mean(), y_test.mean()

### Un modèle simple : la méthode des k plus proches voisins

<img src="./images/knn.png" alt="knn" style="width: 500px;"/>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_model.score(X_test, y_test)

### Un modèle plus avancé : l'arbre de décision

![dtree.png](./images/dtree.png)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=10)
dt_model.fit(X_train, y_train)
dt_model.score(X_test, y_test)

L'arbre de décision présente l'intérêt d'être interprétable :

In [ ]:
from io import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

def plot_dt(dt, max_depth):
    dot_data = StringIO()
    export_graphviz(dt, out_file=dot_data, filled=True, rounded=True, feature_names=features, max_depth=max_depth)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    return Image(graph.create_png())
    
plot_dt(dt_model, max_depth=3)

### S'appuyer sur un ensemble de modèles

Et si on utilisait plutôt une forêt pleine d'arbres ?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, max_depth=8)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)

### La notion d'hyperparamètres

Le concept d'**hyperparamètres** : des paramètres que le modèle ne peut pas apprendre. Par exemple :
- la profondeur de l'arbre `max_depth`
- le nombre d'arbres dans la forêt `n_estimators`
- le nombre de voisins à considérer `n_neighbors`

Comment les optimiser ? En testant !

In [ ]:
scores = []
steps = range(1, 51, 2)
for n in steps:
  repetitions = range(10)
  step_score = []
  for i in repetitions:
    model = DecisionTreeClassifier(max_depth=n)
    model.fit(X_train, y_train)
    step_score.append(model.score(X_test, y_test))
  scores.append(np.mean(step_score))
plt.plot(steps, scores)
plt.title("Justesse en fonction de la profondeur de l'arbre")
plt.show()

### Les réseaux de neurones

<img src="./images/ann.jpg" alt="knn" style="width: 500px;"/>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn_model = Sequential([
              Dense(124, name='hidden', activation='relu', input_shape=(len(features),)),
              Dense(1, name='output', activation='sigmoid'),
           ])

nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
history = nn_model.fit(X_train_norm, y_train, batch_size=200, epochs=100, validation_data=(X_test_norm, y_test))

### La notion d'overfitting

On peut visualiser l'évolution de la performance de lu réseau de neurones au cours de l'entraînement :

In [ ]:
plt.plot(history.history['accuracy'], label='Justesse (train)')
plt.plot(history.history['val_accuracy'], label='Justesse (test)')
plt.title("Evolution de la performance pendant l'entraînement")
plt.legend()
plt.plot()

L'importante notion d'overfitting : lorsque l'algorithme "colle trop" aux données d'entraînement.

## 4. Tester et valider le modèle

On prédit en utilisant le modèle, sur les données de test non vues en entraînement



In [ ]:
y_pred = rf_model.predict(X_test)
y_fake = np.random.choice([0, 1], size=len(y_test), p=[1 - y_test.mean(), y_test.mean()])

### Matrice de confusion



In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, y_pred), index=['Vrai label 0', 'Vrai label 1'], columns=['Label prédit 0', 'Label prédit 1'])

### Utiliser différentes métriques

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

res = []
for metric in [accuracy_score, precision_score, recall_score, f1_score]:
  res.append([metric.__name__, metric(y_test, y_pred), metric(y_test, y_fake)])
pd.DataFrame(res, columns = ['Métrique', 'Score du modèle', "Score de l'aléatoire"])